In [1]:
# from yf_utils import _2_split_train_val_test, _3_random_slices, _4_lookback_slices
from yf_utils import _5_perf_ranks, _6_grp_tuples_sort_sum
from myUtils import pickle_load

# verbose = False  # True prints more output
verbose = True  # True prints more output

path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"
fp_df_close_clean = 'df_close_clean'
fp_rtn_results = 'rtn_results'
df_close_clean = pickle_load(path_data_dump, fp_df_close_clean)

In [2]:
# number of days to lookback from last df date for performance evaluation
days_lookbacks = [15, 30]
days_lookbacks.sort()  # sort list of integers in ascending order

# number of the most-common symbols from days_lookbacks' performance rankings to keep
n_top_syms = 20  

syms_start = 0  # slice start of set_most_common_syms
syms_end = 5  #  slice end of set_most_common_syms

In [3]:
# load past mean daily return of top_set_syms verus SPY return 
rtn_results = pickle_load(path_data_dump, fp_rtn_results)

In [4]:
# grp_top_set_syms_n_freq = []  # list of lists of top_set_symbols_n_freq, there are n_samples lists in list
# grp_top_set_syms = []  # list of lists of top_set_symbols, there are n_samples lists in list
grp_most_common_syms = []  # accumulates the best performing symbols from each days_lookback
grp_perf_ranks = []  # accumulates the best performance ranking symbols from each days_lookback 
grp_dates = []  # dates of days_lookbacks 

for d_lookback in days_lookbacks:
  df = df_close_clean.iloc[-d_lookback:]
  perf_ranks, most_common_syms = _5_perf_ranks(df, n_top_syms=n_top_syms)  # get the best performance symbols and rankings
  grp_most_common_syms.append(most_common_syms)
  grp_perf_ranks.append(perf_ranks)  
  grp_dates.append(df.index[0].strftime("%Y-%m-%d")) 

grp_dates.append(df.index[-1].strftime("%Y-%m-%d"))  # get df's last date
grp_dates.sort()

In [5]:
# flatten list of lists of (symbol:frequency)
flat_grp_most_common_syms = [val for sublist in grp_most_common_syms for val in sublist]
# group symbols from set of days_lookbacks (i.e. lb_slices) and sum frequency of the symbols
set_most_common_syms = _6_grp_tuples_sort_sum(flat_grp_most_common_syms, reverse=True)
# get the top few most-frequent symbol:frequency pairs
top_set_syms_n_freq = set_most_common_syms[0:n_top_syms]
# get symbols from top_set_syms_n_freq
top_set_syms = [i[0] for i in top_set_syms_n_freq[syms_start:syms_end]]

In [6]:
print(f'grp_dates: {grp_dates}')
print(f'top_set_syms_n_freq: {top_set_syms_n_freq}')
print(f'top_set_syms: {top_set_syms}')

grp_dates: ['2023-01-06', '2023-01-30', '2023-02-17']
top_set_syms_n_freq: [('CTLT', 6), ('TA', 6), ('OEC', 5), ('THR', 5), ('TSLA', 5), ('WIRE', 5), ('AMCX', 4), ('AN', 4), ('CAR', 4), ('SMG', 4), ('TNK', 4), ('CRUS', 3), ('ELF', 3), ('EVRI', 3), ('FCF', 3), ('PAG', 3), ('RVNC', 3), ('STNG', 3), ('ACLS', 2), ('ASC', 2)]
top_set_syms: ['CTLT', 'TA', 'OEC', 'THR', 'TSLA']


In [7]:
# import webbrowser

# y_quote = 'https://finance.yahoo.com/quote/'
# for sym in top_set_syms:
#   url = y_quote + sym
#   print(url)
#   webbrowser.open(url)  # open symbols in yahoo finance

In [8]:
# # Importing Pandas to create DataFrame
# import pandas as pd

# my_cols = ['date', 'days_lookbacks', 'n_top_syms', 'syms_start', 'syms_end', 'grp_dates', 'top_set_syms_n_freq', 'top_set_syms']
# # Creating Empty DataFrame and Storing it in variable df
# df = pd.DataFrame(columns=my_cols)

In [9]:
from myUtils import pickle_load, pickle_dump

path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"
fp_top_set_syms = 'df_top_set_syms'  # df to store top set symbols

In [10]:
df = pickle_load(path_data_dump, fp_top_set_syms)
df

,date,days_lookbacks,n_top_syms,syms_start,syms_end,grp_dates,top_set_syms_n_freq,top_set_syms
0,2023-02-10,"[15, 30]",20,0,5,"[2022-12-29, 2023-01-23, 2023-02-10]","[(CRUS, 6), (CTLT, 6), (THR, 5), (ALGN, 4), (I...","[CRUS, CTLT, THR, ALGN, ICPT]"
1,2023-02-13,"[15, 30]",20,0,5,"[2022-12-30, 2023-01-24, 2023-02-13]","[(CRUS, 6), (CTLT, 6), (THR, 5), (ICPT, 4), (L...","[CRUS, CTLT, THR, ICPT, LRN]"
2,2023-02-14,"[15, 30]",20,0,5,"[2023-01-03, 2023-01-25, 2023-02-14]","[(CRUS, 6), (CTLT, 6), (TSLA, 5), (CAR, 4), (O...","[CRUS, CTLT, TSLA, CAR, ON]"
3,2023-02-15,"[15, 30]",20,0,5,"[2023-01-04, 2023-01-26, 2023-02-15]","[(CTLT, 6), (WIRE, 6), (TNK, 5), (TSLA, 5), (C...","[CTLT, WIRE, TNK, TSLA, CAR]"
4,2023-02-16,"[15, 30]",20,0,5,"[2023-01-05, 2023-01-27, 2023-02-16]","[(CTLT, 6), (TA, 6), (CRUS, 5), (THR, 5), (WIR...","[CTLT, TA, CRUS, THR, WIRE]"


In [11]:
row_add = [grp_dates[-1], days_lookbacks, n_top_syms, syms_start, syms_end, grp_dates, top_set_syms_n_freq, top_set_syms]
df.loc[len(df)] = row_add
df

,date,days_lookbacks,n_top_syms,syms_start,syms_end,grp_dates,top_set_syms_n_freq,top_set_syms
0,2023-02-10,"[15, 30]",20,0,5,"[2022-12-29, 2023-01-23, 2023-02-10]","[(CRUS, 6), (CTLT, 6), (THR, 5), (ALGN, 4), (I...","[CRUS, CTLT, THR, ALGN, ICPT]"
1,2023-02-13,"[15, 30]",20,0,5,"[2022-12-30, 2023-01-24, 2023-02-13]","[(CRUS, 6), (CTLT, 6), (THR, 5), (ICPT, 4), (L...","[CRUS, CTLT, THR, ICPT, LRN]"
2,2023-02-14,"[15, 30]",20,0,5,"[2023-01-03, 2023-01-25, 2023-02-14]","[(CRUS, 6), (CTLT, 6), (TSLA, 5), (CAR, 4), (O...","[CRUS, CTLT, TSLA, CAR, ON]"
3,2023-02-15,"[15, 30]",20,0,5,"[2023-01-04, 2023-01-26, 2023-02-15]","[(CTLT, 6), (WIRE, 6), (TNK, 5), (TSLA, 5), (C...","[CTLT, WIRE, TNK, TSLA, CAR]"
4,2023-02-16,"[15, 30]",20,0,5,"[2023-01-05, 2023-01-27, 2023-02-16]","[(CTLT, 6), (TA, 6), (CRUS, 5), (THR, 5), (WIR...","[CTLT, TA, CRUS, THR, WIRE]"
5,2023-02-17,"[15, 30]",20,0,5,"[2023-01-06, 2023-01-30, 2023-02-17]","[(CTLT, 6), (TA, 6), (OEC, 5), (THR, 5), (TSLA...","[CTLT, TA, OEC, THR, TSLA]"


In [12]:
df = df.drop_duplicates(subset=['date'])

In [13]:
pickle_dump(df, path_data_dump, fp_top_set_syms)

In [14]:
_df = pickle_load(path_data_dump, fp_top_set_syms)
_df

,date,days_lookbacks,n_top_syms,syms_start,syms_end,grp_dates,top_set_syms_n_freq,top_set_syms
0,2023-02-10,"[15, 30]",20,0,5,"[2022-12-29, 2023-01-23, 2023-02-10]","[(CRUS, 6), (CTLT, 6), (THR, 5), (ALGN, 4), (I...","[CRUS, CTLT, THR, ALGN, ICPT]"
1,2023-02-13,"[15, 30]",20,0,5,"[2022-12-30, 2023-01-24, 2023-02-13]","[(CRUS, 6), (CTLT, 6), (THR, 5), (ICPT, 4), (L...","[CRUS, CTLT, THR, ICPT, LRN]"
2,2023-02-14,"[15, 30]",20,0,5,"[2023-01-03, 2023-01-25, 2023-02-14]","[(CRUS, 6), (CTLT, 6), (TSLA, 5), (CAR, 4), (O...","[CRUS, CTLT, TSLA, CAR, ON]"
3,2023-02-15,"[15, 30]",20,0,5,"[2023-01-04, 2023-01-26, 2023-02-15]","[(CTLT, 6), (WIRE, 6), (TNK, 5), (TSLA, 5), (C...","[CTLT, WIRE, TNK, TSLA, CAR]"
4,2023-02-16,"[15, 30]",20,0,5,"[2023-01-05, 2023-01-27, 2023-02-16]","[(CTLT, 6), (TA, 6), (CRUS, 5), (THR, 5), (WIR...","[CTLT, TA, CRUS, THR, WIRE]"
5,2023-02-17,"[15, 30]",20,0,5,"[2023-01-06, 2023-01-30, 2023-02-17]","[(CTLT, 6), (TA, 6), (OEC, 5), (THR, 5), (TSLA...","[CTLT, TA, OEC, THR, TSLA]"


In [15]:
df["top_set_syms"]

0    [CRUS, CTLT, THR, ALGN, ICPT]
1     [CRUS, CTLT, THR, ICPT, LRN]
2      [CRUS, CTLT, TSLA, CAR, ON]
3     [CTLT, WIRE, TNK, TSLA, CAR]
4      [CTLT, TA, CRUS, THR, WIRE]
5       [CTLT, TA, OEC, THR, TSLA]
Name: top_set_syms, dtype: object

In [16]:
prev_top_set_syms = df["top_set_syms"].iloc[-2]
prev_top_set_syms

['CTLT', 'TA', 'CRUS', 'THR', 'WIRE']

In [17]:
import pandas as pd
# pd.options.display.width = 120
pd.options.display.max_colwidth = 120

print(f'_df["top_set_syms_n_freq"]:\n{_df["top_set_syms_n_freq"]}\n')
l_last_top_set_syms_n_freq = _df['top_set_syms_n_freq'].iloc[-1]

print(f'_df["top_set_syms"]:\n{_df["top_set_syms"]}')
l_last_top_set_syms = _df['top_set_syms'].iloc[-1]
print(f'last row of top_set_syms:\n{l_last_top_set_syms}')

_df["top_set_syms_n_freq"]:
0    [(CRUS, 6), (CTLT, 6), (THR, 5), (ALGN, 4), (ICPT, 4), (LRN, 4), (TSLA, 4), (AXP, 3), (CINF, 3), (ELF, 3), (EVRI, 3)...
1    [(CRUS, 6), (CTLT, 6), (THR, 5), (ICPT, 4), (LRN, 4), (TSLA, 4), (ALGN, 3), (BSIG, 3), (CINF, 3), (ELF, 3), (EVRI, 3...
2    [(CRUS, 6), (CTLT, 6), (TSLA, 5), (CAR, 4), (ON, 4), (ALGN, 3), (ELF, 3), (FCF, 3), (MPWR, 3), (RVNC, 3), (SMG, 3), ...
3    [(CTLT, 6), (WIRE, 6), (TNK, 5), (TSLA, 5), (CAR, 4), (CRUS, 4), (DCP, 4), (ELF, 3), (FCF, 3), (MPWR, 3), (PH, 3), (...
4    [(CTLT, 6), (TA, 6), (CRUS, 5), (THR, 5), (WIRE, 5), (CAR, 4), (SMG, 4), (TNK, 4), (CINF, 3), (DCP, 3), (ELF, 3), (M...
5    [(CTLT, 6), (TA, 6), (OEC, 5), (THR, 5), (TSLA, 5), (WIRE, 5), (AMCX, 4), (AN, 4), (CAR, 4), (SMG, 4), (TNK, 4), (CR...
Name: top_set_syms_n_freq, dtype: object

_df["top_set_syms"]:
0    [CRUS, CTLT, THR, ALGN, ICPT]
1     [CRUS, CTLT, THR, ICPT, LRN]
2      [CRUS, CTLT, TSLA, CAR, ON]
3     [CTLT, WIRE, TNK, TSLA, CAR]
4      [CTLT, TA, C

In [18]:
recent_top_set_syms = []
for symbols in _df["top_set_syms"]:
  recent_top_set_syms.append(symbols)
recent_top_set_syms

# flatten symbols_download_err which is a list-of-lists
recent_top_set_syms = [val for sublist in recent_top_set_syms for val in sublist]
recent_top_set_syms
# get list of unique symbols in df_OHLCV, list(df) is a list of tuples
# e.g.: [('AAPL', 'Open')..('AAPL', 'Volume'),...
#        ('ZZZZ', 'Open')..('ZZZZ', 'Volume')]
recent_top_set_syms = list(set(recent_top_set_syms))
recent_top_set_syms.append("SPY")
recent_top_set_syms.sort()
print(recent_top_set_syms)


['ALGN', 'CAR', 'CRUS', 'CTLT', 'ICPT', 'LRN', 'OEC', 'ON', 'SPY', 'TA', 'THR', 'TNK', 'TSLA', 'WIRE']


In [19]:
# my_cols.sort() will not change order of l_last_top_set_syms 
my_cols = recent_top_set_syms.copy()  
my_cols.sort()
# my_cols.append('SPY')
_df = df_close_clean[my_cols].tail(10)
# print(_df)
print(_df.pct_change())
_df.pct_change().tail(1)[0:10]

                ALGN       CAR      CRUS      CTLT      ICPT       LRN  \
Date                                                                     
2023-02-06       NaN       NaN       NaN       NaN       NaN       NaN   
2023-02-07  0.007621  0.074627  0.041809  0.068358  0.050823  0.001677   
2023-02-08 -0.029675  0.003067 -0.014440 -0.003772 -0.044219 -0.025359   
2023-02-09 -0.040170 -0.034813 -0.001237  0.009676  0.004337  0.015955   
2023-02-10 -0.017332 -0.027832 -0.004572 -0.021389  0.000960  0.005074   
2023-02-13  0.029923  0.026441  0.015694  0.001277 -0.011026  0.025481   
2023-02-14  0.019246  0.106576 -0.002732 -0.002835  0.028114  0.004688   
2023-02-15  0.020723  0.010615  0.001512 -0.017910 -0.016030  0.012599   
2023-02-16 -0.036112  0.002474 -0.011697  0.040527 -0.057020  0.009677   
2023-02-17 -0.029003 -0.025728 -0.011835 -0.007233  0.017785  0.001597   

                 OEC        ON       SPY        TA       THR       TNK  \
Date                                 

,ALGN,CAR,CRUS,CTLT,ICPT,LRN,OEC,ON,SPY,TA,THR,TNK,TSLA,WIRE
Date,,,,,,,,,,,,,,
2023-02-17,-0.029003,-0.025728,-0.011835,-0.007233,0.017785,0.001597,0.099037,-0.026982,-0.002498,-0.002487,-0.0037,-0.026411,0.031033,0.008794


In [20]:
# today's return from previous top_set_syms
prev_top_set_syms_n_SPY = prev_top_set_syms + ['SPY']
rtn_prev_top_set_syms = _df.pct_change().tail(1)[prev_top_set_syms_n_SPY]
rtn_prev_top_set_syms_mean = rtn_prev_top_set_syms.mean(axis=1)[0]
rtn_prev_top_set_syms_std = rtn_prev_top_set_syms.std(axis=1)[0]
# print(f'rtn_prev_top_set_syms:\n{rtn_prev_top_set_syms}')
print(f'rtn_prev_top_set_syms_mean:                                                {rtn_prev_top_set_syms_mean:>10.6f}')
print(f'rtn_prev_top_set_syms_std:                                                 {rtn_prev_top_set_syms_std:>10.6f}\n')

rtn_SPY = _df.pct_change().tail(1)['SPY'][0]  # [0] changes pd.Series to a value
rtn_dif = rtn_prev_top_set_syms_mean - rtn_SPY
print(f'rtn_SPY:                                                                   {rtn_SPY:>10.6f}\n')                  
print(f"Difference in daily return between yesterday's top_set_syms_mean and SPY:  {rtn_dif:>10.6f}")

print("\nToday's return of previous top_set_syms and SPY")
rtn_prev_top_set_syms

rtn_prev_top_set_syms_mean:                                                 -0.003160
rtn_prev_top_set_syms_std:                                                   0.006869

rtn_SPY:                                                                    -0.002498

Difference in daily return between yesterday's top_set_syms_mean and SPY:   -0.000662

Today's return of previous top_set_syms and SPY


,CTLT,TA,CRUS,THR,WIRE,SPY
Date,,,,,,
2023-02-17,-0.007233,-0.002487,-0.011835,-0.0037,0.008794,-0.002498


In [21]:
_date = rtn_prev_top_set_syms.tail(1).index.strftime('%Y-%m-%d')[0]
_rtn_tss_mean = rtn_prev_top_set_syms_mean
_rtn_tss_std = rtn_prev_top_set_syms_std
_rtn_SPY = rtn_SPY
_rtn_dif = rtn_dif

In [24]:
row_add = [_date, _rtn_tss_mean, _rtn_tss_std, _rtn_SPY, _rtn_dif]

In [25]:
# my_cols
row_add

['2023-02-17',
 -0.003160016245483288,
 0.006869153929946905,
 -0.002498258589011204,
 -0.000661757656472084]

In [26]:
# # # Creating Empty DataFrame and Storing it in variable df
# my_cols = ['date', 'rtn_tss_mean', 'rtn_tss_std', 'rtn_SPY', 'rtn_dif']
# rtn_results = pd.DataFrame(columns=my_cols)

rtn_results.loc[len(rtn_results)] = row_add
rtn_results

,date,rtn_tss_mean,rtn_tss_std,rtn_SPY,rtn_dif
0,2023-02-16,-0.001084,0.033205,-0.013769,0.012684
1,2023-02-17,-0.003160,0.006869,-0.002498,-0.000662


In [27]:
rtn_results = rtn_results.drop_duplicates(subset=['date'])
rtn_results

,date,rtn_tss_mean,rtn_tss_std,rtn_SPY,rtn_dif
0,2023-02-16,-0.001084,0.033205,-0.013769,0.012684
1,2023-02-17,-0.003160,0.006869,-0.002498,-0.000662


In [28]:
pickle_dump(rtn_results, path_data_dump, fp_rtn_results)

In [31]:
_df = pickle_load(path_data_dump, fp_rtn_results)
_df.sort_values(by='date', ascending=False)

,date,rtn_tss_mean,rtn_tss_std,rtn_SPY,rtn_dif
1,2023-02-17,-0.003160,0.006869,-0.002498,-0.000662
0,2023-02-16,-0.001084,0.033205,-0.013769,0.012684
